<a href="https://colab.research.google.com/github/hj2222222/API/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import os
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras import optimizers
from tensorflow.keras.layers import LSTM, Dropout, Dense, Conv1D, MaxPooling1D, Flatten
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

scaler = MinMaxScaler()

stock_data = pd.read_csv('/content/drive/MyDrive/산업 AI 캡스톤/DATA/Preprocessing_Data/Preprocessing_Neumeric_Data/neumeric_preprocess_kospi.csv')
stock_data

In [ ]:
def preprocess_numeric_data(df):

    # 데이터들 소수점 3자리까지 조절
    df = df.round(3)

    # 종가들을 제외하고 스케일링을 진행하기위해서 우선 칼럼들의 이름을 뽑아내기
    df_names = df.columns.tolist()

    # 종가 열(Kospi_close, Kosdaq_close)을 데이터프레임에서 제외
    col_to_drop = [col for col in df.columns if 'Kospi_close' in col or 'Kosdaq_close' in col]
    df_col = df.columns.tolist()
    # 날짜 열(Date)을 데이터프레임에서 제외
    df_col.remove('Date')
    x_col = [item for item in df_col if item not in col_to_drop]

    # MinMax Scaling 적용
    columns_to_scale = x_col  # 스케일링을 적용할 열 목록
    df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

    return df

In [ ]:
# 데이터 전처리
data = stock_data['Kospi_close'].values.reshape(-1, 1)
# stock_data = preprocess_numeric_data(stock_data)
stock_data = stock_data.drop(['Kospi_close'],axis=1)
stock_data = stock_data.drop(['Date'],axis=1)
# scaler = MinMaxScaler()
# data = scaler.fit_transform(data)

In [ ]:
name = [stock_data.columns]

close = ['Kospi_open', 'Kospi_high', 'Kospi_low', 'Kospi_vol', 'dji_open',
       'dji_high', 'dji_low', 'dji_vol', 'us500_open',
       'us500_high', 'us500_low', 'us500_vol', 'ex_AM_open',
       'ex_AM_high', 'ex_AM_low', 'ex_JP_open', 'ex_JP_high',
       'ex_JP_low', 'ko_interest', 'ko_consumer',
       'ko_real_estate']

close_only = ['Kospi_open', 'Kospi_high', 'Kospi_low', 'Kospi_vol', 'dji_open',
       'dji_high', 'dji_low','dji_vol', 'us500_open',
       'us500_high', 'us500_low', 'us500_vol', 'ex_AM_open',
       'ex_AM_high', 'ex_AM_low', 'ex_JP_open', 'ex_JP_high',
       'ex_JP_low']

stock_data_close = stock_data.drop(close, axis=1)
stock_data_close_only = stock_data.drop(close_only, axis=1)

In [ ]:
stock_data_close_only.tail(5)

In [ ]:
# 데이터 분할
train_size = int(len(data) * 0.8)
train_data, test_data = data[:train_size], data[train_size:]
train_data_t, test_data_t = stock_data[:train_size], stock_data[train_size:]

def create_dataset(dataset, data, look_back=10):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(dataset[i:(i + look_back)])
        y.append(data[i + look_back])
    return np.array(X), np.array(y)

look_back = 10  # 몇 일 동안의 데이터를 사용할 것인지 설정

trainX, trainY = create_dataset(train_data_t, train_data, look_back)
testX, testY = create_dataset(test_data_t,test_data, look_back)
print(trainX.shape)
print(trainY.shape)

In [ ]:
earlystopping = EarlyStopping(monitor='val_loss', patience=20)

In [ ]:
# 모델 생성
model = Sequential()
# model.add(Bidirectional(LSTM(units=1024, activation='relu', return_sequences=True), input_shape=(trainX.shape[1], trainX.shape[2])))
# model.add(Dropout(0.1))
model.add(GRU(units=1024, activation='relu', return_sequences=True, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dropout(0.1))
model.add(GRU(units=512, activation='relu', return_sequences=True))
model.add(Dropout(0.1))
model.add(GRU(units=256, activation='relu', return_sequences=True))
model.add(Dropout(0.1))
model.add(GRU(units=128, activation='relu',return_sequences=True))
model.add(Dropout(0.1))
model.add(GRU(units=64, activation='relu',return_sequences=True))
model.add(Dropout(0.1))
model.add(GRU(units=32, activation='relu'))
model.add(Dropout(0.1))
# model.add(GRU(units=16, activation='relu'))
# model.add(Dropout(0.1))
model.add(Dense(units=1))
model.summary()

model.compile(optimizer='adam', loss='mean_squared_error')


# 모델 학습
history = model.fit(trainX, trainY, epochs=300, batch_size=64,
          validation_split=0.1, verbose=2)            #callbacks=[early_stopping]
model.save_weights('./lstm_weights.h1')

plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()
plt.show()

# 예측
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# 예측 결과 역 스케일링
# trainPredict = trainPredict.detach().cpu().numpy()
# trainPredict = scaler.inverse_transform(trainPredict)
# trainY = trainY.detach().cpu().numpy()
# trainY = scaler.inverse_transform([trainY])
# testPredict = testPredict.detach().cpu().numpy()
# testPredict = scaler.inverse_transform(testPredict)
# testY = testY.detach().cpu().numpy()
# testY = scaler.inverse_transform([testY])

# 예측 시각화
plt.plot(trainY, label='Actual Price')
plt.plot(trainPredict, label='Predicted Price')
plt.legend()
plt.title('Training Data')
plt.show()

plt.plot(testY, label='Actual Price')
plt.plot(testPredict, label='Predicted Price')
plt.legend()
plt.title('Testing Data')
plt.show()

In [ ]:
# # 데이터 분할
# train_size = int(len(data) * 0.8)
# train_data, test_data = data[:train_size], data[train_size:]
# train_data_t, test_data_t = stock_data_close[:train_size], stock_data_close[train_size:]

# def create_dataset(dataset, data, look_back=10):
#     X, y = [], []
#     for i in range(len(data) - look_back):
#         X.append(dataset[i:(i + look_back)])
#         y.append(data[i + look_back])
#     return np.array(X), np.array(y)

# look_back = 10  # 몇 일 동안의 데이터를 사용할 것인지 설정

# trainX, trainY = create_dataset(train_data_t, train_data, look_back)
# testX, testY = create_dataset(test_data_t,test_data, look_back)
# print(trainX.shape)
# print(trainY.shape)

In [ ]:
# 모델 생성
model = Sequential()
# model.add(GRU(units=1024, activation='relu', return_sequences=True, input_shape=(trainX.shape[1], trainX.shape[2])))
# model.add(Dropout(0.2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(GRU(units=512, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=256, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=128, activation='relu',return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=64, activation='relu',return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=32, activation='relu',return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.summary()


loss = Huber()
optimizer = Adam(0.0005)
model.compile(loss=Huber(), optimizer=optimizer, metrics=['mse'])

filename = os.path.join('tmp', 'ckeckpointer.ckpt')
checkpoint = ModelCheckpoint(filename,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1)

# model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
history = model.fit(trainX, trainY, epochs=300, batch_size=64,
          validation_split=0.1, verbose=2,callbacks=[checkpoint, earlystopping])

model.save_weights('./lstm_weights.h2')

model.load_weights(filename)

plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()
plt.show()

# 예측
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# 예측 결과 역 스케일링
# trainPredict = trainPredict.detach().cpu().numpy()
# trainPredict = scaler.inverse_transform(trainPredict)
# trainY = trainY.detach().cpu().numpy()
# trainY = scaler.inverse_transform([trainY])
# testPredict = testPredict.detach().cpu().numpy()
# testPredict = scaler.inverse_transform(testPredict)
# testY = testY.detach().cpu().numpy()
# testY = scaler.inverse_transform([testY])

# 예측 시각화
plt.plot(trainY, label='Actual Price')
plt.plot(trainPredict, label='Predicted Price')
plt.legend()
plt.title('Training Data')
plt.show()

plt.plot(testY, label='Actual Price')
plt.plot(testPredict, label='Predicted Price')
plt.legend()
plt.title('Testing Data')
plt.show()

In [ ]:
# # 데이터 분할
# train_size = int(len(data) * 0.8)
# train_data, test_data = data[:train_size], data[train_size:]
# train_data_t, test_data_t = stock_data_close_only[:train_size], stock_data_close_only[train_size:]

# def create_dataset(dataset, data, look_back=10):
#     X, y = [], []
#     for i in range(len(data) - look_back):
#         X.append(dataset[i:(i + look_back)])
#         y.append(data[i + look_back])
#     return np.array(X), np.array(y)

# look_back = 10  # 몇 일 동안의 데이터를 사용할 것인지 설정

# trainX, trainY = create_dataset(train_data_t, train_data, look_back)
# testX, testY = create_dataset(test_data_t,test_data, look_back)
# print(trainX.shape)
# print(trainY.shape)

In [ ]:
# 모델 생성
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(units=1024, activation='relu', return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=512, activation='relu', return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=256, activation='relu', return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=128, activation='relu',return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=64, activation='relu',return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=32, activation='relu',return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=16, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=1))
model.summary()

loss = Huber()
optimizer = Adam(0.0005)
model.compile(loss=Huber(), optimizer=optimizer, metrics=['mse'])

filename = os.path.join('tmp', 'ckeckpointer.ckpt')
checkpoint = ModelCheckpoint(filename,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1)

# model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
history = model.fit(trainX, trainY, epochs=300, batch_size=64,
          validation_split=0.1, verbose=2,callbacks=[checkpoint, earlystopping])

model.save_weights('./lstm_weights.h3')

model.load_weights(filename)

plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()
plt.show()

# 예측
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# 예측 결과 역 스케일링
# trainPredict = trainPredict.detach().cpu().numpy()
# trainPredict = scaler.inverse_transform(trainPredict)
# trainY = trainY.detach().cpu().numpy()
# trainY = scaler.inverse_transform([trainY])
# testPredict = testPredict.detach().cpu().numpy()
# testPredict = scaler.inverse_transform(testPredict)
# testY = testY.detach().cpu().numpy()
# testY = scaler.inverse_transform([testY])

# 예측 시각화
plt.plot(trainY, label='Actual Price')
plt.plot(trainPredict, label='Predicted Price')
plt.legend()
plt.title('Training Data')
plt.show()

plt.plot(testY, label='Actual Price')
plt.plot(testPredict, label='Predicted Price')
plt.legend()
plt.title('Testing Data')
plt.show()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 8, 64)             7168      
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 4, 64)             0         
 g1D)                                                            
                                                                 
 lstm_7 (LSTM)               (None, 4, 1024)           4460544   
                                                                 
 dropout_73 (Dropout)        (None, 4, 1024)           0         
                                                                 
 lstm_8 (LSTM)               (None, 4, 512)            3147776   
                                                                 
 dropout_74 (Dropout)        (None, 4, 512)            0         
                                                     

In [ ]:
#양방향 lstm + 코드의 깊이 증가 및 유닛의 수 증가로 바꿔주는 코드


# model = Sequential()
# model.add(Bidirectional(LSTM(units=1024, activation='relu', return_sequences=True), input_shape=(trainX.shape[1], trainX.shape[2]))
# model.add(Dropout(0.1))
# model.add(Bidirectional(LSTM(units=512, activation='relu', return_sequences=True))
# model.add(Dropout(0.1))
# model.add(Bidirectional(LSTM(units=256, activation='relu', return_sequences=True))
# model.add(Dropout(0.1))
# model.add(Bidirectional(LSTM(units=128, activation='relu',return_sequences=True))
# model.add(Dropout(0.1))
# model.add(Bidirectional(LSTM(units=64, activation='relu',return_sequences=True))
# model.add(Dropout(0.1))
# model.add(Bidirectional(LSTM(units=32, activation='relu',return_sequences=True))
# model.add(Dropout(0.1))
# model.add(Bidirectional(LSTM(units=16, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(units=1))
# model.summary()

In [ ]:
# model = Sequential()

# Add a Convolutional layer
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]))
# model.add(MaxPooling1D(pool_size=2)

# # Add LSTM layers
# model.add(LSTM(units=1024, activation='relu', return_sequences=True))
# model.add(Dropout(0.1))
# model.add(LSTM(units=512, activation='relu', return_sequences=True))
# model.add(Dropout(0.1))
# model.add(LSTM(units=256, activation='relu', return_sequences=True))
# model.add(Dropout(0.1))
# model.add(LSTM(units=128, activation='relu', return_sequences=True))
# model.add(Dropout(0.1))
# model.add(LSTM(units=64, activation='relu', return_sequences=True))
# model.add(Dropout(0.1))
# model.add(LSTM(units=32, activation='relu', return_sequences=True))
# model.add(Dropout(0.1))
# model.add(LSTM(units=16, activation='relu'))
# model.add(Dropout(0.1))

# # Add the output Dense layer
# model.add(Dense(units=1))

# model.summary()

In [ ]:
# Sequence 학습에 비교적 좋은 퍼포먼스를 내는 Huber()를 사용합니다.
loss = Huber()
optimizer = Adam(0.0005)
model.compile(loss=Huber(), optimizer=optimizer, metrics=['mse'])

filename = os.path.join('tmp', 'ckeckpointer.ckpt')
checkpoint = ModelCheckpoint(filename,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1)

history = model.fit(train_data,
                    validation_data=(test_data),
                    epochs=50,
                    callbacks=[checkpoint, earlystopping])

model.load_weights(filename)